In [1]:
from sklearn import svm, metrics, random_projection, cross_validation, datasets, grid_search, linear_model, metrics, random_projection, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.linear_model import SGDRegressor
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.svm import SVC
import xgboost as xgb
%pylab inline

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
dataframe = pd.read_csv('dataframe/genres_40.csv')
labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
dataframe.loc[:, 'm1':] = preprocessing.normalize(dataframe.loc[:, 'm1':])
dataframe = dataframe.sample(frac=1)

shape(dataframe)

(1601, 201)

In [5]:
def search_gr(data, label, list_model, list_param_gr):
    cv = cross_validation.StratifiedShuffleSplit(label, n_iter = 5, test_size = 0.2, random_state = 1)
    i = 1
    for model, param_gr in zip(list_model, list_param_gr):
        grid_cv = grid_search.GridSearchCV(model, param_gr, cv = cv)
        grid_cv.fit(data, label)
        print('model' + str(i), grid_cv.best_score_, grid_cv.best_params_)

In [6]:
#kn

In [7]:
parameters_knc =[
    {
        'n_neighbors': [5, 7, 17, 27],
        'metric': ['manhattan', 'minkowski', 'euclidean']
    }
]
knc = KNeighborsClassifier(weights = 'distance')

parameters_lrc =[
    {
        'C':  [400, 430, 440, 450, 500]
    }
]
lrc = LogisticRegression(random_state=1)

parameters_svcc =[
    {
        'C': [1, 5, 10, 15, 17],
        'gamma': [1, 3, 5, 10, 15],
        'kernel': ['rbf', 'linear', 'poly']
    }
]
svcc = SVC(random_state=1)



In [8]:
data = np.hstack((load('knc.npy'), load('svcc.npy'), load('lrc.npy'), load('xgbc.npy')))

In [9]:
search_gr(data, load('labels.npy'), [svcc, knc, lrc], [parameters_svcc, parameters_knc, parameters_lrc])

model1 0.8794117647058823 {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
model1 0.8838235294117647 {'metric': 'manhattan', 'n_neighbors': 27}
model1 0.861764705882353 {'C': 400}


In [109]:
lrc = LogisticRegression(C = 430, random_state=1)
ridge_scoring2 = cross_validation.cross_val_score(lrc, dataframe.loc[:, 'm1':], dataframe['tag'], 
                                                          scoring='f1_weighted', cv = 5)
print('Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring2.mean(), ridge_scoring2.max(),
                                                     ridge_scoring2.min(), ridge_scoring2.std()))

Ridge mean:0.7862337148051285, max:0.8033071830373378, min:0.771793555351703, std:0.010949127566336057


In [4]:
def search_abnom(dataframe, label, g, nu):
    svm_ = svm.OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for label in labels:
    svm_label = search_abnom(dataframe, label, 2, 0.15).tolist()
    dataframe[dataframe['tag'] == label] = dataframe[dataframe['tag'] == label][np.array(svm_label) == 1]
    dataframe = dataframe.dropna()
print(shape(dataframe))

(1352, 201)


In [107]:
lrc = LogisticRegression(C = 430, random_state=1)
ridge_scoring2 = cross_validation.cross_val_score(lrc, dataframe.loc[:, 'm1':], dataframe['tag'], 
                                                          scoring='f1_weighted', cv = 5)
print('Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring2.mean(), ridge_scoring2.max(),
                                                     ridge_scoring2.min(), ridge_scoring2.std()))

Ridge mean:0.8120383930038099, max:0.8470242155267215, min:0.7769197646114958, std:0.030620005516038183


In [12]:
search_gr(dataframe.loc[:, 'm1':], dataframe['tag'], [svcc], [parameters_svcc])

model1 0.8776223776223776 {'C': 5, 'gamma': 10, 'kernel': 'rbf'}


In [10]:
parameters_rfc =[
    {
        'max_depth':[20, 23, 25, 30], 
        'n_estimators':[1000, 1300, 1500, 1600]
    }
]
rfc = RandomForestClassifier(random_state=1)

parameters_xgbc =[
    {
        'max_depth': [2], 
        'n_estimators': [1800, 2000, 4000, 6000] 
    }
]
xgbc = xgb.XGBClassifier(silent=False, learning_rate=0.009)


In [ ]:
search_gr(data, load('labels.npy'), [rfc], [parameters_rfc])

model1 0.8772058823529412 {'max_depth': 20, 'n_estimators': 1300}


In [ ]:
search_gr(data, load('labels.npy'), [xgbc], [parameters_xgbc])

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
